# Main script to call functions from other scripts

In [ ]:
# Import libraries

In [2]:
# Initialize other notebooks
# ------------ Initialize preprocessing script
%run "Preprocessing.ipynb"

## Full pipeline
Extracting edited part --> Lemmatization, stemming and removing stopwords --> TF-IDF --> Feed to model --> Return prediction

In [3]:
globals()['models_loaded'] = False

# ------------ Replace this with a trained ML model
def predict(df):
    if any([x in df.diff.lower() for x in ['bad', 'lol', 'joke']]):
        return 'vandal'
    else:
        return 'safe'
# ------------ Replace this with a trained ML model

predict_udf = udf(predict, StringType())

def process(time, rdd):
    """""""""
    This function will serve as the construction pipeline.
    Here all preprocessing functions need to be called and 
    need to be able to handle one wiki edit.
    """""""""
    if rdd.isEmpty():
        return
    
    print("========= %s =========" % str(time))
    
    #########################################################
    # PREPROCESSING & FEATURE ENGINEERING
    #########################################################
    df = spark.read.json(rdd)
    
    clean_df = get_clean_df(df)
    df_with_difference = get_difference_column(clean_df)
    final_df = split_difference_into_removed_added(df_with_difference)
    final_df.show(3)
    
    #########################################################
    # MAKE PREDICTION
    #########################################################
    
#     # Utilize our predict function
#     df_withpreds = df_withdiff.withColumn("pred", predict_udf(
#         struct([df_withdiff[x] for x in df_withdiff.columns])
#     ))
#     df_withpreds.show()
    
#     # Normally, you wouldn't use a UDF (User Defined Function) Python function to predict (you can)
#     # But an MLlib model you've built and saved with Spark
#     # In this case, you need to prevent loading your model in every call to "process" as follows:
    
#     # Load in the model if not yet loaded:
#     if not globals()['models_loaded']:
#         # load in your models here
#         globals()['my_model'] = '***' # Replace '***' with:    [...].load('my_logistic_regression')
#         globals()['models_loaded'] = True
        
#     # And then predict using the loaded model: 
#     # df_result = globals()['my_model'].transform(df)
#     # df_result.show()

## Begin online stream and make prediction per wiki edit

In [4]:
# Start the prediction task
ssc = StreamingContext(sc, 10) # ---------------------> Get minibatches every 10 seconds
lines = ssc.socketTextStream("seppe.net", 7778)
lines.foreachRDD(process) # --------------------------> Perform the function 'process' on every RDD
ssc_t = StreamingThread(ssc)
ssc_t.start()

========= 2020-04-30 09:59:00 =========
+-----+------------+--------------------+-----------+--------------------+--------------------+-------------+-----------+
|label|     comment|          title_page|  name_user|      clean_old_text|      clean_new_text|removed_words|added_words|
+-----+------------+--------------------+-----------+--------------------+--------------------+-------------+-----------+
| safe|active cases|2020 coronavirus ...|maranello10|pp-protectedsmall...|pp-protectedsmall...|  nd ha n't  |       or h|
+-----+------------+--------------------+-----------+--------------------+--------------------+-------------+-----------+



In [5]:
# Stop the prediction task
ssc_t.stop()

----- Stopping... this may take a few seconds -----
========= 2020-04-30 09:59:10 =========
+-----+---------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+
|label|        comment|          title_page|name_user|      clean_old_text|      clean_new_text|       removed_words|         added_words|
+-----+---------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+
| safe|election result|abdullah ahmad ba...|     nctb|short description...|short description...|rowspan3'''p35 ke...|rowspan9'''kepala...|
| safe|          empty|    guangxi massacre| screditc|the '''guangxi ma...|the '''guangxi ma...| during the bolua...|               empty|
+-----+---------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+

